New notebook to test the difference between the tech rec mongo db and gal alum.
Currently, code working for gal alum but not tech recruiters.
1. Copy progress from mongodb_test
2. Replicate the gal alum process from
* Connection to mongo db
* Pull data into df
* Manipulate to correct format
* Create csv of all companies
* Create csvs of individual companies

3. New notebook section for tech recruiters

In [1]:
import pandas as pd
import mongo
import ast
import os

# Galvanize Alumni CSV build

In [3]:
mongo.connect_mongo()
mongo.connect_coll('gal_part_proj', 'gal_alum')
cursor = mongo.coll.find({}, {'_id':0})
df = pd.DataFrame(cursor)
df = df.T

In [4]:
index = list()
results = list()

for idx, row in df.iterrows():
    index.append(idx)
    results.append(row[row.notnull()].values[0])

clean_df = pd.DataFrame(results, index=index)
clean_df.rename({0: 'Scrape Results'}, axis=1, inplace=True)
clean_df.head()

,Scrape Results
University of Texas at Austin,People Link 404
Apple,{'Erin Gong': 'https://www.linkedin.com/in/eri...
Dell Technologies,{'Daniel Donovan': 'https://www.linkedin.com/i...
IBM,{'James Helfrich': 'https://www.linkedin.com/i...
National Instruments,No results


# Austin Technical Recruiters CSV build
## Load data from mongo

In [5]:
mongo.connect_mongo()
mongo.connect_coll('gal_part_proj', 'ATX_tech_rec')
cursor = mongo.coll.find({}, {'_id':0})

df_tr = pd.DataFrame(cursor)
df_tr = df_tr.T

## Austin Tech Recruiters pipeline error corrections
My dataframe pipeline is failing for the tech recruiters mongo pull. Successively passing the Alumni pull with the same process. Steps to troubleshoot proceed.

### Issue: Pandas takes the assumption of the data series for the DataFrame based upon the __first item__ in the list/array being built. The first item in the list/array was a string. 

### Fix: Force-inserting an item into the list remedied this. Line 9 & 10 below.

In [6]:
# pipeline for ATX Tech Recruiters
index_tr = list()
results_tr = list()

for idx, row in df_tr.iterrows():
    index_tr.append(idx)
    results_tr.append(row[row.notnull()].values[0])

results_tr.insert(0, 'ignore')
index_tr.insert(0, 'ignore')    
clean_df_tr = pd.DataFrame(results_tr, index=index_tr)
clean_df_tr.drop('ignore', axis=0, inplace=True)
clean_df_tr.rename({0: 'Scrape Results'}, axis=1, inplace=True)
clean_df_tr.head()

,Scrape Results
9Gauge Partners,{'Carrie Guillory': 'https://www.linkedin.com/...
Abbott,{'Brandy Williams Broden': 'https://www.linked...
Aceable,"{'Andrew Bolisay (he, him, his)': 'https://www..."
AdbaKx,Company Page 404
Addison Group,{'Hattie Cox': 'https://www.linkedin.com/in/ha...


# Convert DataFrame to CSVs for each company (observations)

## Create local directory for csv
### Variables should be switched between `tech_rec` and `alumni` for saving between the two types of variables.

In [34]:
''' change below assignment for saving different CSVs'''
contacts = clean_df_tr['Scrape Results'] #clean_df
employee_type = 'tech_rec' #alumni

In [43]:
'''Make Directories'''
# os.mkdir('../data/alumni')
os.mkdir('../data/tech_rec')

In [45]:
# dict_flag = contacts.apply(lambda x: isinstance(x, dict))
# contacts['Scrape Results'] = contacts['Scrape Results'].apply(lambda x: ast.literal_eval(x))
contacts = contacts[dict_flag] #clean_df
employee_type = 'tech_rec' #alumni

for i, c in zip(contacts.index, contacts):
    co_contacts = pd.DataFrame(c.values(), index=c.keys())
    co_contacts.to_csv('../data/{0}/{1}.csv'.format(employee_type, i))

# gDrive interaction testing

In [111]:
import gspread

# Scrape tech recruiters work locations for database
1. Concat CSVs
2. c-p to a new script and test out small samples from list of tech recs
 - How many recruiters do I have? 5661
    - script takes at least 6s to complete each search ~ 9 hours to complete
3. script flow,
 - open LI
 - togo tech rec link

In [2]:
import pandas as pd

In [3]:
import glob

In [4]:
path = '../data/tech_rec' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

In [5]:
len(all_files)

121

In [6]:
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, names=['recruiter', 'url'])
    co_name = filename
    co_name = co_name.split('/')[-1]
    co_name = co_name[:-4]
    df['co_name'] = co_name
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [7]:
frame

,recruiter,url,co_name
0,"Chelsea Johnson, CSP",https://www.linkedin.com/in/chelsea-johnson-cs...,Greystar
1,Christina T Jackson,https://www.linkedin.com/in/christinatjackson/,Greystar
2,Heather Patton,https://www.linkedin.com/in/heatherhaynes/,Greystar
3,Kristin Doherty,https://www.linkedin.com/in/recruiter-kristin/,Greystar
4,Kacy Fowler,https://www.linkedin.com/in/kacyjenkins/,Greystar
...,...,...,...
28303,Jennifer Cosenza,https://www.linkedin.com/in/jennifer-cosenza/,REX
28304,Chrystina Wyatt,https://www.linkedin.com/in/chrystina-wyatt-48...,REX
28305,Sam B,https://www.linkedin.com/in/sam-b-b57931123/,REX
28306,Ronald Harris Jr,https://www.linkedin.com/in/ronald-harris-jr-3...,REX


In [8]:
frame[frame['co_name']=='REX']

,recruiter,url,co_name
28300,Josh Decker,https://www.linkedin.com/in/joshadecker/,REX
28301,Frankie Brochu,https://www.linkedin.com/in/frankie-brochu-b93...,REX
28302,Jeremy Chapa,https://www.linkedin.com/in/jeremychapa/,REX
28303,Jennifer Cosenza,https://www.linkedin.com/in/jennifer-cosenza/,REX
28304,Chrystina Wyatt,https://www.linkedin.com/in/chrystina-wyatt-48...,REX
28305,Sam B,https://www.linkedin.com/in/sam-b-b57931123/,REX
28306,Ronald Harris Jr,https://www.linkedin.com/in/ronald-harris-jr-3...,REX
28307,Chay Samudrala,https://www.linkedin.com/in/chay-samudrala-903...,REX


In [9]:
frame.head(10)

,recruiter,url,co_name
0,"Chelsea Johnson, CSP",https://www.linkedin.com/in/chelsea-johnson-cs...,Greystar
1,Christina T Jackson,https://www.linkedin.com/in/christinatjackson/,Greystar
2,Heather Patton,https://www.linkedin.com/in/heatherhaynes/,Greystar
3,Kristin Doherty,https://www.linkedin.com/in/recruiter-kristin/,Greystar
4,Kacy Fowler,https://www.linkedin.com/in/kacyjenkins/,Greystar
5,Jessica Bravo,https://www.linkedin.com/in/jessica-bravo-40a0...,Greystar
6,Kevin Cure,https://www.linkedin.com/in/kevincure/,Greystar
7,Antonio Rogers,https://www.linkedin.com/in/antonio-rogers-6ba...,Greystar
8,Jaclyn Howard,https://www.linkedin.com/in/jaclynhoward/,Greystar
9,Kellie Kieserman,https://www.linkedin.com/in/kellie-kieserman-7...,Greystar


In [25]:
frame['location'] = ''

In [26]:
frame.head(1)

,recruiter,url,co_name,location
0,"Chelsea Johnson, CSP",https://www.linkedin.com/in/chelsea-johnson-cs...,Greystar,


In [10]:
frame_sample = frame.sample(10)
frame_sample

,recruiter,url,co_name
17550,Robert Half,NaN,_techrecruiters_with_location
8351,The Home Depot,NaN,_techrecruiters_with_location
18557,Merck,NaN,_techrecruiters_with_location
13526,Ford Motor Company,NaN,_techrecruiters_with_location
4295,keefiat shah,https://www.linkedin.com/in/keefiat-shah-9ba4b...,Pyramid Consulting
9195,EY,NaN,_techrecruiters_with_location
26105,EY,NaN,_techrecruiters_with_location
3208,Keith Ohlstrom,https://www.linkedin.com/in/keith-ohlstrom-316...,EY
2108,TJ Looney,https://www.linkedin.com/in/t-j-looney-0a15bb9a/,Global Resources
21911,Aditi Consulting,NaN,_techrecruiters_with_location


In [11]:
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import chromedriver_binary
from bs4 import BeautifulSoup

In [12]:
def get_login():
    '''
    Access keys from external file and placed in a list.

    Parameters
    ----------
    None:

    Returns
    ----------
    creds: (list)
        Return keys used for session.
    '''
    f = open('../data/LI_login.txt', 'r')
    creds = f.readlines()

    for idx, key in enumerate(creds):
        creds[idx] = key.replace('\n', '')

    return creds

In [13]:
def login():
    '''
    Login into LinkedIn and webdriver session for more web manipulation.
    Keys generated from get_login() function.

    Parameters
    ----------
    None:

    Returns
    ----------
    driver: (selenium.webdriver.chrome.webdriver.WebDriver)
        Return webdriver session for web manipulation.
    '''
    # session keys for LI instance
    email, pw = get_login()
    
    # selenium webdriver
    driver = webdriver.Chrome()
    driver.get('https://www.linkedin.com/')
    # log in
    sleep(2)
    driver.find_element_by_id('session_key').send_keys(email)
    sleep(1)
    driver.find_element_by_id('session_password').send_keys(pw+Keys.RETURN)

    return driver

In [18]:
def scrape_location(driver, url):
    '''scrape tech recruiters location info'''
    sleep(3)
    driver.get(url)
    r = driver.page_source
    soup = BeautifulSoup(r, 'html.parser')
    flex_card = soup.find('div', 'flex-1 mr5')
    location = flex_card.find('li', 't-16 t-black t-normal inline-block')
    location = location.text.lstrip().rstrip()
    sleep(3)

    return location




In [19]:
driver = login()

# location = frame_sample['url'].apply(lambda x: scrape_location(x))
# test notes
# 1. more sleeps, it goes to fast between sites
# 2. close driver


In [100]:
location

4429                             Delhi, India
1906      Grand Prairie, Texas, United States
5262                     Greater Phoenix Area
4262       Alpharetta, Georgia, United States
2619    The Hague, South Holland, Netherlands
824          Cedar Park, Texas, United States
160     Minneapolis, Minnesota, United States
322         New York, New York, United States
452           New York City Metropolitan Area
360      San Antonio, Texas Metropolitan Area
Name: url, dtype: object

In [102]:
frame_sample['location'] = location

In [107]:
frame_sample.head(2)

,recruiter,url,co_name,location
4429,Rawat Gaurav,https://www.linkedin.com/in/rawat-gaurav-1a523...,Pyramid Consulting,"Delhi, India"
1906,Emmanuel Paul,https://www.linkedin.com/in/emmanuel-paul-b371...,Global Resources,"Grand Prairie, Texas, United States"


In [114]:
# !ls ../data/tech_rec -al

In [112]:
frame_sample.to_csv('../data/tech_rec/techrecruiters_with_location.csv')

In [33]:
for idx, row in frame[0:3].iterrows():
        location = scrape_location(driver, row['url'])
        frame['location'][idx] = location

In [43]:
frame.iloc[idx]

recruiter                                Heather Patton
url          https://www.linkedin.com/in/heatherhaynes/
co_name                                        Greystar
location         Lake Forest, California, United States
Name: 2, dtype: object

In [34]:
location

'Lake Forest, California, United States'

In [44]:
row.index

Index(['recruiter', 'url', 'co_name', 'location'], dtype='object')

In [35]:
frame.head(3)

,recruiter,url,co_name,location
0,"Chelsea Johnson, CSP",https://www.linkedin.com/in/chelsea-johnson-cs...,Greystar,"Phoenix, Arizona, United States"
1,Christina T Jackson,https://www.linkedin.com/in/christinatjackson/,Greystar,Miami-Fort Lauderdale Area
2,Heather Patton,https://www.linkedin.com/in/heatherhaynes/,Greystar,"Lake Forest, California, United States"


In [37]:
import numpy as np

In [45]:
df = pd.DataFrame(columns=frame.columns)

In [46]:
df

,recruiter,url,co_name,location


In [50]:
df = df.append(row)

In [53]:
df.to_csv('../data/test.csv', index=False)

In [51]:
df

,recruiter,url,co_name,location
2,Heather Patton,https://www.linkedin.com/in/heatherhaynes/,Greystar,"Lake Forest, California, United States"


In [54]:
test = frame.head(3)
test

,recruiter,url,co_name,location
0,"Chelsea Johnson, CSP",https://www.linkedin.com/in/chelsea-johnson-cs...,Greystar,"Phoenix, Arizona, United States"
1,Christina T Jackson,https://www.linkedin.com/in/christinatjackson/,Greystar,Miami-Fort Lauderdale Area
2,Heather Patton,https://www.linkedin.com/in/heatherhaynes/,Greystar,"Lake Forest, California, United States"


In [56]:
df[1:].to_csv('../data/test.csv', index=False, header=False, mode='a')